# Fine-tuning DINOv3 on Animals-10 Dataset

This notebook demonstrates how to fine-tune a DINOv3 model on the [Animals-10 dataset](https://www.kaggle.com/datasets/alessiocorrado99/animals10/data) from Kaggle.

**Dataset Overview:**
- **Classes**: 10 animal categories (dog, cat, horse, spider, butterfly, chicken, sheep, cow, squirrel, elephant)
- **Images**: ~26,000 total images
- **Format**: ImageFolder structure
- **Use Case**: Multi-class animal classification

**What we'll cover:**
1. Dataset download and exploration
2. Data preprocessing and validation
3. Configuration generation
4. Model training with multiple paradigms
5. Evaluation and comparison
6. Model deployment

## Setup and Dependencies

In [ ]:
import os
import sys
from pathlib import Path
import shutil
import zipfile
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import classification_report, confusion_matrix
import json
import yaml
from datetime import datetime

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Set up directories
data_dir = project_root / "data" / "animals10"
config_dir = project_root / "configs"
output_dir = project_root / "outputs" / "animals10_experiment"

# Create directories
data_dir.mkdir(parents=True, exist_ok=True)
config_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Data directory: {data_dir}")
print(f"GPU available: {torch.cuda.is_available()}")

## Step 1: Dataset Download and Preparation

First, we'll download the Animals-10 dataset from Kaggle. You'll need to have the Kaggle API set up:

```bash
pip install kaggle
# Place your kaggle.json in ~/.kaggle/
```

In [ ]:
# Download dataset using Kaggle API
def download_animals10_dataset(data_dir):
    """Download Animals-10 dataset from Kaggle."""
    
    # Check if already downloaded
    if (data_dir / "raw-img").exists():
        print("Dataset already exists!")
        return
    
    print("Downloading Animals-10 dataset from Kaggle...")
    
    # Download using Kaggle API
    import kaggle
    
    try:
        kaggle.api.dataset_download_files(
            'alessiocorrado99/animals10',
            path=data_dir,
            unzip=True
        )
        print("Download completed!")
        
    except Exception as e:
        print(f"Kaggle API error: {e}")
        print("Please ensure you have Kaggle API set up correctly.")
        print("Alternative: Download manually from https://www.kaggle.com/datasets/alessiocorrado99/animals10/data")
        return
    
    # List downloaded files
    print("\nDataset structure:")
    for item in data_dir.rglob("*"):
        if item.is_dir():
            print(f"📁 {item.relative_to(data_dir)}")

# Download the dataset
download_animals10_dataset(data_dir)

In [ ]:
# Explore dataset structure
def explore_dataset(data_path):
    """Explore the dataset structure and statistics."""
    
    raw_img_path = data_path / "raw-img"
    
    if not raw_img_path.exists():
        print(f"Dataset not found at {raw_img_path}")
        return
    
    print("=== Dataset Exploration ===")
    
    # Count images per class
    class_counts = {}
    total_images = 0
    
    for class_dir in raw_img_path.iterdir():
        if class_dir.is_dir():
            # Count images in class directory
            image_count = len(list(class_dir.glob("*.jpg")))
            class_counts[class_dir.name] = image_count
            total_images += image_count
    
    print(f"Total images: {total_images:,}")
    print(f"Number of classes: {len(class_counts)}")
    print("\nClass distribution:")
    
    # Sort by count
    sorted_classes = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)
    
    for class_name, count in sorted_classes:
        percentage = (count / total_images) * 100
        print(f"  {class_name:<12}: {count:>5,} images ({percentage:>5.1f}%)")
    
    return class_counts, total_images

class_counts, total_images = explore_dataset(data_dir)

In [ ]:
# Visualize class distribution
if class_counts:
    plt.figure(figsize=(12, 6))
    
    classes = list(class_counts.keys())
    counts = list(class_counts.values())
    
    # Create bar plot
    bars = plt.bar(classes, counts, color='skyblue', edgecolor='navy', alpha=0.7)
    plt.title('Animals-10 Dataset: Class Distribution', fontsize=16, fontweight='bold')
    plt.xlabel('Animal Classes', fontsize=12)
    plt.ylabel('Number of Images', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    
    # Add count labels on bars
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
                f'{count:,}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.grid(axis='y', alpha=0.3)
    plt.show()
    
    # Calculate statistics
    counts_array = np.array(counts)
    print(f"\nDataset Statistics:")
    print(f"  Mean images per class: {counts_array.mean():.0f}")
    print(f"  Std images per class: {counts_array.std():.0f}")
    print(f"  Min images per class: {counts_array.min():,}")
    print(f"  Max images per class: {counts_array.max():,}")
    print(f"  Class balance ratio: {counts_array.min() / counts_array.max():.2f}")

In [ ]:
# Sample a few images from each class
def visualize_sample_images(data_path, num_samples=2):
    """Visualize sample images from each class."""
    
    raw_img_path = data_path / "raw-img"
    
    if not raw_img_path.exists():
        return
    
    classes = [d.name for d in raw_img_path.iterdir() if d.is_dir()]
    classes.sort()
    
    fig, axes = plt.subplots(len(classes), num_samples, 
                            figsize=(num_samples * 3, len(classes) * 2.5))
    fig.suptitle('Sample Images from Each Class', fontsize=16, fontweight='bold')
    
    for i, class_name in enumerate(classes):
        class_dir = raw_img_path / class_name
        image_files = list(class_dir.glob("*.jpg"))[:num_samples]
        
        for j, img_path in enumerate(image_files):
            try:
                img = Image.open(img_path).convert('RGB')
                
                ax = axes[i, j] if len(classes) > 1 else axes[j]
                ax.imshow(img)
                ax.set_title(f"{class_name}\n{img.size[0]}x{img.size[1]}")
                ax.axis('off')
                
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    
    plt.tight_layout()
    plt.show()

# Visualize samples
visualize_sample_images(data_dir, num_samples=3)

## Step 2: Data Preprocessing and Validation

Now we'll preprocess the data using our framework's preprocessing tools.

In [ ]:
# Import our preprocessing tools
from data.preprocessing import ImagePreprocessor
from data.validation import DatasetValidator
from data.dataset import create_dataset
from data.augmentations import create_transforms

# Run data validation and quality analysis
def analyze_dataset_quality(data_path):
    """Analyze dataset quality using our framework tools."""
    
    raw_img_path = data_path / "raw-img"
    
    print("=== Dataset Quality Analysis ===")
    
    # Initialize validator
    validator = DatasetValidator()
    
    # Validate dataset structure
    validation_results = validator.validate_imagefolder_dataset(
        raw_img_path,
        min_samples_per_class=100,  # Minimum 100 samples per class
        validate_images=True
    )
    
    print(f"Dataset valid: {validation_results['is_valid']}")
    print(f"Total samples: {validation_results['total_samples']:,}")
    print(f"Number of classes: {validation_results['num_classes']}")
    
    if validation_results['corrupted_images']:
        print(f"⚠️  Found {len(validation_results['corrupted_images'])} corrupted images")
        for img_path in validation_results['corrupted_images'][:5]:  # Show first 5
            print(f"  - {img_path}")
    else:
        print("✅ No corrupted images found")
    
    # Analyze class balance
    balance_info = validator.analyze_class_balance(raw_img_path)
    print(f"\nClass balance ratio: {balance_info['balance_ratio']:.3f}")
    print(f"Dataset balanced: {balance_info['is_balanced']}")
    
    return validation_results

# Run quality analysis
validation_results = analyze_dataset_quality(data_dir)

In [ ]:
# Preprocess the dataset using our framework
def preprocess_animals10_dataset(input_dir, output_dir):
    """Preprocess Animals-10 dataset for training."""
    
    raw_img_path = input_dir / "raw-img"
    processed_path = output_dir / "processed"
    
    if processed_path.exists():
        print("Processed dataset already exists!")
        return processed_path
    
    print("Preprocessing Animals-10 dataset...")
    
    # Initialize preprocessor
    preprocessor = ImagePreprocessor(
        target_size=(224, 224),
        resize_method="resize_shortest",
        quality_threshold=50,  # Filter low quality images
        convert_grayscale_to_rgb=True
    )
    
    processed_path.mkdir(parents=True, exist_ok=True)
    
    total_processed = 0
    total_failed = 0
    
    # Process each class
    for class_dir in tqdm(raw_img_path.iterdir(), desc="Processing classes"):
        if not class_dir.is_dir():
            continue
            
        class_name = class_dir.name
        output_class_dir = processed_path / class_name
        output_class_dir.mkdir(exist_ok=True)
        
        # Process images in class
        image_files = list(class_dir.glob("*.jpg"))
        
        for img_path in tqdm(image_files, desc=f"Processing {class_name}", leave=False):
            try:
                # Process image
                processed_img = preprocessor.process_image(img_path)
                
                if processed_img is not None:
                    # Save processed image
                    output_path = output_class_dir / img_path.name
                    processed_img.save(output_path, 'JPEG', quality=95)
                    total_processed += 1
                else:
                    total_failed += 1
                    
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                total_failed += 1
    
    print(f"\nPreprocessing completed:")
    print(f"  Successfully processed: {total_processed:,} images")
    print(f"  Failed to process: {total_failed:,} images")
    print(f"  Success rate: {total_processed/(total_processed+total_failed)*100:.1f}%")
    
    return processed_path

# Preprocess the dataset
processed_data_path = preprocess_animals10_dataset(data_dir, data_dir)

In [ ]:
# Split dataset into train/val/test sets
def split_animals10_dataset(processed_path, output_dir):
    """Split the processed dataset into train/val/test sets."""
    
    train_dir = output_dir / "train"
    val_dir = output_dir / "val"
    test_dir = output_dir / "test"
    
    if train_dir.exists():
        print("Dataset already split!")
        return train_dir, val_dir, test_dir
    
    print("Splitting dataset into train/val/test sets...")
    
    # Create split directories
    for split_dir in [train_dir, val_dir, test_dir]:
        split_dir.mkdir(parents=True, exist_ok=True)
    
    # Split ratios
    train_ratio = 0.7  # 70% for training
    val_ratio = 0.15   # 15% for validation
    test_ratio = 0.15  # 15% for testing
    
    np.random.seed(42)  # For reproducible splits
    
    split_stats = {}
    
    # Process each class
    for class_dir in processed_path.iterdir():
        if not class_dir.is_dir():
            continue
            
        class_name = class_dir.name
        
        # Create class directories in each split
        for split_dir in [train_dir, val_dir, test_dir]:
            (split_dir / class_name).mkdir(exist_ok=True)
        
        # Get all images and shuffle
        image_files = list(class_dir.glob("*.jpg"))
        np.random.shuffle(image_files)
        
        # Calculate split indices
        n_images = len(image_files)
        n_train = int(n_images * train_ratio)
        n_val = int(n_images * val_ratio)
        
        # Split files
        train_files = image_files[:n_train]
        val_files = image_files[n_train:n_train + n_val]
        test_files = image_files[n_train + n_val:]
        
        # Copy files to respective directories
        for files, split_dir in [(train_files, train_dir), (val_files, val_dir), (test_files, test_dir)]:
            for img_path in files:
                dest_path = split_dir / class_name / img_path.name
                shutil.copy2(img_path, dest_path)
        
        split_stats[class_name] = {
            'total': n_images,
            'train': len(train_files),
            'val': len(val_files),
            'test': len(test_files)
        }
    
    # Print split statistics
    print("\nDataset split completed:")
    
    total_stats = {'train': 0, 'val': 0, 'test': 0}
    
    for class_name, stats in split_stats.items():
        print(f"  {class_name:<12}: Train={stats['train']:>4}, Val={stats['val']:>3}, Test={stats['test']:>3}")
        for split in total_stats:
            total_stats[split] += stats[split]
    
    print(f"\nTotal counts:")
    for split, count in total_stats.items():
        percentage = count / sum(total_stats.values()) * 100
        print(f"  {split.capitalize()}: {count:,} ({percentage:.1f}%)")
    
    return train_dir, val_dir, test_dir

# Split the dataset
train_dir, val_dir, test_dir = split_animals10_dataset(processed_data_path, data_dir)

## Step 3: Configuration Generation

Let's generate different configurations for various training paradigms.

In [ ]:
# Generate configurations for different training paradigms
def generate_animals10_configs():
    """Generate different configurations for Animals-10 training."""
    
    base_config = {
        "name": "animals10_experiment",
        "description": "DINOv3 fine-tuning on Animals-10 dataset",
        "seed": 42,
        
        "model": {
            "variant": "dinov3_vitb16",
            "task_type": "classification",
            "num_classes": 10,
            "pretrained": True,
            "dropout": 0.1
        },
        
        "data": {
            "image_size": [224, 224],
            "batch_size": 32,
            "num_workers": 4,
            "pin_memory": True,
            "shuffle_train": True,
            "train_data_path": str(train_dir),
            "val_data_path": str(val_dir)
        },
        
        "optimizer": {
            "name": "adamw",
            "betas": [0.9, 0.999],
            "eps": 1e-8
        },
        
        "scheduler": {
            "name": "cosine_annealing",
            "min_lr": 1e-6,
            "warmup_type": "linear"
        },
        
        "augmentation": {
            "domain": "natural",
            "train": {
                "horizontal_flip": True,
                "color_jitter": {
                    "brightness": 0.2,
                    "contrast": 0.2,
                    "saturation": 0.2,
                    "hue": 0.1
                },
                "random_rotation": 15,
                "gaussian_blur": 0.1
            },
            "val": {
                "resize_shortest": True,
                "center_crop": True,
                "normalize": True
            }
        },
        
        "logging": {
            "log_interval": 10,
            "save_interval": 5,
            "use_tensorboard": True,
            "use_wandb": False  # Set to True if you have W&B account
        },
        
        "evaluation": {
            "metrics": ["accuracy", "top5_accuracy", "f1_macro"],
            "save_predictions": True,
            "visualize_predictions": True
        }
    }
    
    # Configuration variants
    configs = {}
    
    # 1. Linear Probing (fast training)
    linear_probe_config = base_config.copy()
    linear_probe_config.update({
        "name": "animals10_linear_probe",
        "training": {
            "mode": "linear_probe",
            "epochs": 50,
            "learning_rate": 1e-3,
            "weight_decay": 0.01,
            "warmup_epochs": 5,
            "freeze_backbone": True,
            "mixed_precision": True,
            "gradient_clipping": 1.0,
            "early_stopping": {
                "patience": 10,
                "min_delta": 1e-4
            }
        }
    })
    configs['linear_probe'] = linear_probe_config
    
    # 2. Full Fine-tuning (best performance)
    full_finetune_config = base_config.copy()
    full_finetune_config.update({
        "name": "animals10_full_finetune",
        "training": {
            "mode": "full_fine_tune",
            "epochs": 30,
            "learning_rate": 5e-5,
            "weight_decay": 0.05,
            "warmup_epochs": 5,
            "freeze_backbone": False,
            "mixed_precision": True,
            "gradient_clipping": 1.0,
            "differential_lr": {
                "backbone_lr_ratio": 0.1,
                "head_lr_ratio": 1.0
            },
            "early_stopping": {
                "patience": 8,
                "min_delta": 1e-4
            }
        }
    })
    configs['full_finetune'] = full_finetune_config
    
    # 3. Lightweight model (for resource-constrained environments)
    lightweight_config = base_config.copy()
    lightweight_config.update({
        "name": "animals10_lightweight",
        "model": {
            **base_config["model"],
            "variant": "dinov3_vits16"  # Smaller model
        },
        "data": {
            **base_config["data"],
            "batch_size": 64  # Larger batch for smaller model
        },
        "training": {
            "mode": "full_fine_tune",
            "epochs": 40,
            "learning_rate": 1e-4,
            "weight_decay": 0.01,
            "warmup_epochs": 5,
            "freeze_backbone": False,
            "mixed_precision": True,
            "gradient_clipping": 1.0,
            "early_stopping": {
                "patience": 10,
                "min_delta": 1e-4
            }
        }
    })
    configs['lightweight'] = lightweight_config
    
    # Save configurations
    for name, config in configs.items():
        config_path = config_dir / f"animals10_{name}.yaml"
        
        with open(config_path, 'w') as f:
            yaml.dump(config, f, default_flow_style=False, indent=2)
        
        print(f"Generated config: {config_path}")
    
    return configs

# Generate configurations
configs = generate_animals10_configs()

# Display configuration summary
print("\n=== Configuration Summary ===")
for name, config in configs.items():
    training = config['training']
    model = config['model']
    
    print(f"\n{name.upper()}:")
    print(f"  Model: {model['variant']}")
    print(f"  Mode: {training['mode']}")
    print(f"  Epochs: {training['epochs']}")
    print(f"  Learning Rate: {training['learning_rate']}")
    print(f"  Batch Size: {config['data']['batch_size']}")
    print(f"  Frozen Backbone: {training.get('freeze_backbone', False)}")

## Step 4: Model Training

Now let's train models using different paradigms. We'll start with linear probing for quick results.

In [ ]:
# Import training modules
from models.model_factory import create_model
from training.trainer import DINOv3Trainer
from utils.schemas import ExperimentConfig
from utils.logging import setup_logging
import logging

# Function to train a model with given configuration
def train_animals10_model(config_name, config, notebook_mode=True):
    """Train Animals-10 model with given configuration."""
    
    print(f"\n=== Training {config_name.upper()} Model ===")
    
    # Create experiment config
    experiment_config = ExperimentConfig(**config)
    
    # Setup logging
    if notebook_mode:
        logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
        logger = logging.getLogger("dinov3_training")
    else:
        logger = setup_logging(
            name="dinov3_training",
            level="INFO",
            log_dir=output_dir / config_name / "logs"
        )
    
    # Create transforms
    transform_manager = create_transforms(
        domain=experiment_config.augmentation.domain,
        image_size=experiment_config.data.image_size,
        train_kwargs=experiment_config.augmentation.train,
        val_kwargs=experiment_config.augmentation.val
    )
    
    # Create datasets
    print("Creating datasets...")
    train_dataset = create_dataset(
        data_path=experiment_config.data.train_data_path,
        annotation_format="imagefolder",
        transform=transform_manager.get_train_transform(),
        cache_images=False
    )
    
    val_dataset = create_dataset(
        data_path=experiment_config.data.val_data_path,
        annotation_format="imagefolder",
        transform=transform_manager.get_val_transform(),
        cache_images=False
    )
    
    print(f"Train dataset: {len(train_dataset):,} samples")
    print(f"Val dataset: {len(val_dataset):,} samples")
    print(f"Classes: {train_dataset.classes}")
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=experiment_config.data.batch_size,
        shuffle=experiment_config.data.shuffle_train,
        num_workers=experiment_config.data.num_workers,
        pin_memory=experiment_config.data.pin_memory,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=experiment_config.data.batch_size,
        shuffle=False,
        num_workers=experiment_config.data.num_workers,
        pin_memory=experiment_config.data.pin_memory,
        drop_last=False
    )
    
    # Create model
    print(f"Creating {experiment_config.model.variant} model...")
    model = create_model(experiment_config.model)
    model.set_training_mode(experiment_config.training.mode)
    
    # Print model info
    model_info = model.get_model_info()
    print(f"Model: {model_info['variant']}")
    print(f"Parameters: {model_info['total_parameters']:,} total, {model_info['trainable_parameters']:,} trainable")
    print(f"Model size: {model_info['model_size_mb']:.1f} MB")
    
    # Create trainer
    trainer = DINOv3Trainer(
        model=model,
        config=experiment_config.training,
        train_dataloader=train_loader,
        val_dataloader=val_loader,
        logger=logger,
        experiment_tracker=None  # Disable external tracking for notebook
    )
    
    # Train model
    experiment_output_dir = output_dir / config_name
    experiment_output_dir.mkdir(exist_ok=True)
    
    print("\nStarting training...")
    start_time = datetime.now()
    
    training_results = trainer.train(experiment_output_dir)
    
    end_time = datetime.now()
    
    # Print results
    print(f"\n=== Training Results ({config_name}) ===")
    print(f"Training time: {end_time - start_time}")
    print(f"Final epoch: {training_results['final_epoch']}")
    if training_results.get('best_metric'):
        print(f"Best validation accuracy: {training_results['best_metric']:.4f}")
    
    return {
        'config_name': config_name,
        'model': model,
        'training_results': training_results,
        'experiment_config': experiment_config,
        'datasets': {'train': train_dataset, 'val': val_dataset},
        'output_dir': experiment_output_dir
    }

# Train linear probe model (fastest)
print("Starting with Linear Probe training for quick results...")
linear_probe_results = train_animals10_model('linear_probe', configs['linear_probe'])

In [ ]:
# Optional: Train full fine-tuning model (better performance, takes longer)
train_full_finetune = input("Train full fine-tuning model? (y/n): ").lower().startswith('y')

if train_full_finetune:
    print("Training full fine-tuning model (this will take longer)...")
    full_finetune_results = train_animals10_model('full_finetune', configs['full_finetune'])
else:
    print("Skipping full fine-tuning (you can run this later)")
    full_finetune_results = None

## Step 5: Model Evaluation and Analysis

Let's evaluate our trained model(s) and analyze their performance.

In [ ]:
# Import evaluation tools
from evaluation.evaluators import create_evaluator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

def evaluate_animals10_model(model_results, test_data_path):
    """Comprehensive evaluation of trained model."""
    
    config_name = model_results['config_name']
    model = model_results['model']
    experiment_config = model_results['experiment_config']
    
    print(f"\n=== Evaluating {config_name.upper()} Model ===")
    
    # Create test dataset
    transform_manager = create_transforms(
        domain=experiment_config.augmentation.domain,
        image_size=experiment_config.data.image_size
    )
    
    test_dataset = create_dataset(
        data_path=str(test_data_path),
        annotation_format="imagefolder",
        transform=transform_manager.get_val_transform(),
        cache_images=False
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    print(f"Test dataset: {len(test_dataset):,} samples")
    
    # Create evaluator
    device = next(model.parameters()).device
    evaluator = create_evaluator(
        task_type=experiment_config.model.task_type,
        num_classes=experiment_config.model.num_classes,
        device=device
    )
    
    # Run evaluation
    model.eval()
    all_predictions = []
    all_targets = []
    all_confidences = []
    
    print("Running evaluation...")
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader, desc="Evaluating")):
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Forward pass
            outputs = model(inputs)
            logits = outputs['logits'] if isinstance(outputs, dict) else outputs
            
            # Get predictions
            predictions = torch.argmax(logits, dim=1)
            confidences = torch.softmax(logits, dim=1).max(dim=1)[0]
            
            # Store results
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_confidences.extend(confidences.cpu().numpy())
            
            # Update evaluator
            evaluator.update(logits.cpu(), targets.cpu())
    
    # Compute metrics
    metrics = evaluator.compute()
    
    # Print results
    print(f"\nTest Results ({config_name}):")
    for metric_name, value in metrics.items():
        if isinstance(value, float):
            print(f"  {metric_name}: {value:.4f}")
    
    # Classification report
    class_names = test_dataset.classes
    print(f"\nDetailed Classification Report:")
    print(classification_report(all_targets, all_predictions, target_names=class_names))
    
    # Confusion matrix
    cm = confusion_matrix(all_targets, all_predictions)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {config_name.title()} Model')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Per-class accuracy
    per_class_acc = cm.diagonal() / cm.sum(axis=1)
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(class_names, per_class_acc, color='skyblue', edgecolor='navy')
    plt.title(f'Per-Class Accuracy - {config_name.title()} Model')
    plt.xlabel('Animal Classes')
    plt.ylabel('Accuracy')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    
    # Add accuracy labels on bars
    for bar, acc in zip(bars, per_class_acc):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Confidence analysis
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(all_confidences, bins=50, alpha=0.7, color='green', edgecolor='black')
    plt.xlabel('Confidence Score')
    plt.ylabel('Frequency')
    plt.title('Confidence Score Distribution')
    plt.grid(alpha=0.3)
    
    plt.subplot(1, 2, 2)
    correct_predictions = np.array(all_predictions) == np.array(all_targets)
    correct_confidences = np.array(all_confidences)[correct_predictions]
    incorrect_confidences = np.array(all_confidences)[~correct_predictions]
    
    plt.hist(correct_confidences, bins=30, alpha=0.7, label='Correct', color='green')
    plt.hist(incorrect_confidences, bins=30, alpha=0.7, label='Incorrect', color='red')
    plt.xlabel('Confidence Score')
    plt.ylabel('Frequency')
    plt.title('Confidence: Correct vs Incorrect')
    plt.legend()
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Save results
    results = {
        'config_name': config_name,
        'test_metrics': metrics,
        'per_class_accuracy': dict(zip(class_names, per_class_acc)),
        'classification_report': classification_report(all_targets, all_predictions, target_names=class_names, output_dict=True),
        'confusion_matrix': cm.tolist(),
        'confidence_stats': {
            'mean': float(np.mean(all_confidences)),
            'std': float(np.std(all_confidences)),
            'mean_correct': float(np.mean(correct_confidences)),
            'mean_incorrect': float(np.mean(incorrect_confidences))
        }
    }
    
    # Save to file
    results_path = model_results['output_dir'] / 'test_results.json'
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\nResults saved to: {results_path}")
    
    return results

# Evaluate the trained model(s)
linear_probe_eval = evaluate_animals10_model(linear_probe_results, test_dir)

if 'full_finetune_results' in locals() and full_finetune_results is not None:
    full_finetune_eval = evaluate_animals10_model(full_finetune_results, test_dir)
else:
    full_finetune_eval = None

In [ ]:
# Compare models (if we have multiple)
if full_finetune_eval is not None:
    print("\n=== Model Comparison ===")
    
    models_comparison = {
        'Linear Probe': linear_probe_eval['test_metrics'],
        'Full Fine-tune': full_finetune_eval['test_metrics']
    }
    
    comparison_df = pd.DataFrame(models_comparison).round(4)
    print(comparison_df)
    
    # Visualization
    metrics_to_compare = ['accuracy', 'f1_macro']
    
    fig, axes = plt.subplots(1, len(metrics_to_compare), figsize=(15, 5))
    
    for idx, metric in enumerate(metrics_to_compare):
        values = [models_comparison[model][metric] for model in models_comparison.keys()]
        
        ax = axes[idx] if len(metrics_to_compare) > 1 else axes
        bars = ax.bar(models_comparison.keys(), values, 
                     color=['lightcoral', 'skyblue'], edgecolor='navy')
        ax.set_title(f'{metric.title()} Comparison')
        ax.set_ylabel(metric.title())
        ax.set_ylim(0, 1)
        
        # Add value labels
        for bar, value in zip(bars, values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                   f'{value:.4f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

else:
    print("\nOnly linear probe model was trained. To compare with full fine-tuning, set train_full_finetune=True above.")

## Step 6: Model Deployment

Let's deploy our best model as an API server for inference.

In [ ]:
# Find the best model
def get_best_model_path(results_list):
    """Find the path to the best trained model."""
    
    available_models = []
    
    if linear_probe_results:
        model_path = linear_probe_results['output_dir'] / 'best_model.pth'
        if model_path.exists():
            available_models.append({
                'name': 'Linear Probe',
                'path': model_path,
                'config': linear_probe_results['output_dir'] / 'config.yaml',
                'accuracy': linear_probe_eval['test_metrics']['accuracy']
            })
    
    if full_finetune_eval is not None:
        model_path = full_finetune_results['output_dir'] / 'best_model.pth'
        if model_path.exists():
            available_models.append({
                'name': 'Full Fine-tune',
                'path': model_path,
                'config': full_finetune_results['output_dir'] / 'config.yaml',
                'accuracy': full_finetune_eval['test_metrics']['accuracy']
            })
    
    if not available_models:
        print("No trained models found!")
        return None
    
    # Select best model
    best_model = max(available_models, key=lambda x: x['accuracy'])
    
    print(f"Best model: {best_model['name']} (Accuracy: {best_model['accuracy']:.4f})")
    print(f"Model path: {best_model['path']}")
    print(f"Config path: {best_model['config']}")
    
    return best_model

best_model_info = get_best_model_path([linear_probe_results, full_finetune_results if 'full_finetune_results' in locals() else None])

In [ ]:
# Test inference with the best model
def test_model_inference(model_path, config_path, test_images_dir):
    """Test inference with a few sample images."""
    
    print("\n=== Testing Model Inference ===")
    
    # Load model and config (this simulates what the API server does)
    print(f"Loading model from: {model_path}")
    
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location='cpu')
    config_dict = checkpoint['config']
    experiment_config = ExperimentConfig(**config_dict)
    
    # Create model
    model = create_model(experiment_config.model)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # Create transforms
    transform_manager = create_transforms(
        domain=experiment_config.augmentation.domain,
        image_size=experiment_config.data.image_size
    )
    val_transform = transform_manager.get_val_transform()
    
    # Get class names
    class_names = checkpoint.get('class_names', [f'class_{i}' for i in range(experiment_config.model.num_classes)])
    
    # Test on a few random images
    test_results = []
    
    # Collect sample images from test set
    sample_images = []
    for class_dir in test_images_dir.iterdir():
        if class_dir.is_dir():
            class_images = list(class_dir.glob("*.jpg"))[:2]  # 2 images per class
            for img_path in class_images:
                sample_images.append((img_path, class_dir.name))
    
    # Randomly select 10 images
    np.random.shuffle(sample_images)
    sample_images = sample_images[:10]
    
    print(f"Testing inference on {len(sample_images)} sample images...")
    
    # Create a visualization
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    axes = axes.flatten()
    
    with torch.no_grad():
        for idx, (img_path, true_class) in enumerate(sample_images):
            # Load and preprocess image
            image = Image.open(img_path).convert('RGB')
            input_tensor = val_transform(image).unsqueeze(0)  # Add batch dimension
            
            # Forward pass
            start_time = time.time()
            outputs = model(input_tensor)
            inference_time = time.time() - start_time
            
            # Get prediction
            logits = outputs['logits'] if isinstance(outputs, dict) else outputs
            probabilities = torch.softmax(logits, dim=1)
            confidence, predicted_idx = torch.max(probabilities, 1)
            
            predicted_class = class_names[predicted_idx.item()]
            confidence_score = confidence.item()
            
            # Store results
            test_results.append({
                'image_path': str(img_path),
                'true_class': true_class,
                'predicted_class': predicted_class,
                'confidence': confidence_score,
                'correct': predicted_class == true_class,
                'inference_time_ms': inference_time * 1000
            })
            
            # Visualize
            ax = axes[idx]
            ax.imshow(image)
            
            # Color code: green for correct, red for incorrect
            color = 'green' if predicted_class == true_class else 'red'
            
            ax.set_title(f'True: {true_class}\nPred: {predicted_class}\nConf: {confidence_score:.3f}\n{inference_time*1000:.1f}ms',
                        fontsize=10, color=color, fontweight='bold')
            ax.axis('off')
    
    plt.tight_layout()
    plt.suptitle('Sample Inference Results', fontsize=16, y=1.02)
    plt.show()
    
    # Print summary
    correct_predictions = sum(1 for r in test_results if r['correct'])
    avg_confidence = np.mean([r['confidence'] for r in test_results])
    avg_inference_time = np.mean([r['inference_time_ms'] for r in test_results])
    
    print(f"\nInference Test Summary:")
    print(f"  Accuracy on samples: {correct_predictions}/{len(test_results)} ({correct_predictions/len(test_results)*100:.1f}%)")
    print(f"  Average confidence: {avg_confidence:.3f}")
    print(f"  Average inference time: {avg_inference_time:.1f} ms")
    
    return test_results

# Test inference
if best_model_info:
    inference_results = test_model_inference(
        best_model_info['path'],
        best_model_info['config'],
        test_dir
    )

In [ ]:
# Instructions for deploying the model as an API
if best_model_info:
    print("\n" + "="*60)
    print("🚀 DEPLOYMENT INSTRUCTIONS")
    print("="*60)
    
    print("\n1. Start the API server:")
    print(f"   MODEL_PATH='{best_model_info['path']}' \\")
    print(f"   CONFIG_PATH='{best_model_info['config']}' \\")
    print(f"   python {project_root}/src/api/server.py")
    
    print("\n2. Test the API:")
    print('   curl -X POST "http://localhost:8000/predict" \\')
    print('     -F "file=@path/to/your/animal/image.jpg" \\')
    print('     -F "confidence_threshold=0.5"')
    
    print("\n3. View API documentation:")
    print('   Open: http://localhost:8000/docs')
    
    print("\n4. Docker deployment:")
    print(f"   docker build -f {project_root}/docker/Dockerfile.deploy -t animals10-api .")
    print(f"   docker run -p 8000:8000 -v {best_model_info['path'].parent}:/models animals10-api")
    
    print("\n5. Kubernetes deployment:")
    print(f"   kubectl apply -f {project_root}/k8s/")
    
    print("\n" + "="*60)
    
    # Save deployment config
    deployment_config = {
        'model_name': 'Animals-10 Classifier',
        'model_path': str(best_model_info['path']),
        'config_path': str(best_model_info['config']),
        'model_type': best_model_info['name'],
        'test_accuracy': best_model_info['accuracy'],
        'classes': [
            'butterfly', 'cat', 'chicken', 'cow', 'dog',
            'elephant', 'horse', 'sheep', 'spider', 'squirrel'
        ],
        'deployment_date': datetime.now().isoformat(),
        'performance_metrics': {
            'avg_inference_time_ms': np.mean([r['inference_time_ms'] for r in inference_results]),
            'avg_confidence': np.mean([r['confidence'] for r in inference_results])
        }
    }
    
    deployment_config_path = output_dir / 'deployment_config.json'
    with open(deployment_config_path, 'w') as f:
        json.dump(deployment_config, f, indent=2)
    
    print(f"\nDeployment configuration saved to: {deployment_config_path}")

## Summary and Next Steps

**What we accomplished:**

1. ✅ **Downloaded and explored** the Animals-10 dataset (26k+ images, 10 classes)
2. ✅ **Preprocessed and validated** the data using our framework tools
3. ✅ **Generated configurations** for different training paradigms
4. ✅ **Trained DINOv3 models** using linear probing (and optionally full fine-tuning)
5. ✅ **Evaluated model performance** with comprehensive metrics and visualizations
6. ✅ **Tested inference** on sample images with timing analysis
7. ✅ **Prepared for deployment** with API server instructions

**Key Results:**
- **Dataset**: 10 animal classes with good class balance
- **Model Performance**: Achieved high accuracy on animal classification
- **Inference Speed**: Fast inference suitable for real-time applications
- **Deployment Ready**: Model ready for production deployment

**Next Steps:**

1. **🚀 Deploy the API**: Follow the deployment instructions above
2. **📱 Build an App**: Create a web or mobile app using the API
3. **🎯 Fine-tune Further**: Try different hyperparameters or model variants
4. **📊 Monitor Performance**: Set up monitoring in production
5. **🔄 Continuous Learning**: Implement feedback loops for model improvement

**Additional Experiments to Try:**

- **Different Model Variants**: Try ViT-L/16 or ViT-S/16
- **Data Augmentation**: Experiment with domain-specific augmentations
- **Model Optimization**: Apply quantization or pruning for faster inference
- **Multi-Label Extension**: Modify for multi-label animal detection
- **Transfer to Related Tasks**: Use the trained features for other animal datasets

---

**🎉 Congratulations!** You've successfully fine-tuned a DINOv3 model for animal classification and prepared it for deployment!